In [10]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import optuna
from hyperopt import hp, fmin, tpe, Trials

# Инициализация данных

In [11]:
data_df = pd.read_csv('data/train_sem09.csv')
data_df.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [12]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3751 entries, 0 to 3750
Columns: 1777 entries, Activity to D1776
dtypes: float64(942), int64(835)
memory usage: 50.9 MB


Проверим, сбалансирован ли датасет по таргету.

In [13]:
data_df['Activity'].value_counts(normalize=True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

Расхождение небольшое, поэтому можно считать, что таргет сбалансирован. Выделим факторы и таргет, а также разобьем выборку на обучающую и тестовую.

In [14]:
X = data_df.drop('Activity', axis=1)
y = data_df['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

Обучим модель логистической регресии.

In [15]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 500, random_state=42)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)
print('F1 на тестовом наборе для базовой модели логистической регресии: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1 на тестовом наборе для базовой модели логистической регресии: 0.79


Обучим модель случайных деревьев.

In [16]:
#Создаем объект класса случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rf.fit(X_train, y_train)
#Выводим значения метрики 

y_test_pred = rf.predict(X_test)
print('F1 на тестовом наборе для базовой модели случайного леса: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

F1 на тестовом наборе для базовой модели случайного леса: 0.83


# GridSearchCV

## Логистическая регрессия

In [17]:
param_grid = [
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
]

grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=500), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_train, y_train) 

y_test_pred = grid_search_1.predict(X_test)
print('F1 на тестовом наборе для логистической регресии (GridSearchCV): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: total: 8.44 s
Wall time: 10min 15s
F1 на тестовом наборе для логистической регресии (GridSearchCV): 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


## Случайный лес

In [18]:
param_grid = {'n_estimators': [100, 200, 300, 400] , # тип регуляризации
              'max_depth': [5, 10, 15, 20], # алгоритм оптимизации
               'min_samples_split': [5, 7, 9]}, # уровень силы регурялизации

grid_search_2 = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

%time grid_search_2.fit(X_train, y_train) 

y_test_pred = grid_search_2.predict(X_test)
print('F1 на тестовом наборе для случайного леса (GridSearchCV): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: total: 8.7 s
Wall time: 2min 57s
F1 на тестовом наборе для случайного леса (GridSearchCV): 0.83
Наилучшие значения гиперпараметров: {'max_depth': 15, 'min_samples_split': 7, 'n_estimators': 400}


## GridSearchCV — выводы

С помощью GridSearchCV нам не удалось улучшить метрику F1 ни одной из модели.

# RandomizedSearchCV

## Логистическая регрессия

In [19]:
param_distributions = [
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'], # алгоритм оптимизации
               'C': list(np.linspace(0.01, 1, 10, dtype=float))}, # уровень силы регурялизации
              
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': list(np.linspace(0.01, 1, 10, dtype=float))}, # уровень силы регурялизации
]
            
random_search_1 = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=500), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  

%time random_search_1.fit(X_train, y_train) 

y_test_pred = random_search_1.predict(X_test)
print('F1 на тестовом наборе для логистической регресии (RandomizedSearchCV): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_1.best_params_))

CPU times: total: 531 ms
Wall time: 2min 1s
F1 на тестовом наборе для логистической регресии (RandomizedSearchCV): 0.80
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.56}


## Случайный лес

In [20]:
param_distributions = {'n_estimators': list(range(100, 401, 100)),
              'min_samples_leaf': list(range(5, 12, 2)),
              'max_depth': list(range(10, 41, 10))
              }
            
random_search_2 = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  

%time random_search_2 .fit(X_train, y_train) 

y_test_pred = random_search_2 .predict(X_test)
print('F1 на тестовом наборе для случайного леса (RandomizedSearchCV): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_2 .best_params_))

CPU times: total: 1.64 s
Wall time: 39.6 s
F1 на тестовом наборе для случайного леса (RandomizedSearchCV): 0.83
Наилучшие значения гиперпараметров: {'n_estimators': 100, 'min_samples_leaf': 5, 'max_depth': 20}


## RandomizedSearchCV — выводы

Удалось увеличить метрику F1 только для логистической регрессии.

# Hyperopt

## Логистическая регрессия

In [29]:
# зададим пространство поиска гиперпараметров
pen = ['l2', 'none']
sol = ['lbfgs', 'sag']

space ={
    'penalty': hp.choice('penalty', ['l2', 'none']) , # тип регуляризации
    'solver': hp.choice('solver', ['lbfgs', 'sag']), # алгоритм оптимизации
    'C': hp.uniform('C', 0.01, 10) # уровень силы регурялизации
} 
      
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': str(params['penalty']), 
              'solver': str(params['solver']), 
             'C': float(params['C'])
              }
  
    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=42, max_iter=500)

    # обучаем модель
    # model.fit(X, y)
    # score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

trials_lr = Trials() # используется для логирования результатов

best = fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_lr, # логирование результатов
          rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    random_state=42,
    max_iter=1000,
    penalty=pen[best['penalty']],
    solver=sol[best['solver']],
    C=float(best['C'])
)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('F1 на тестовом наборе для логистической регресии (Hyperopt): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

100%|██████████| 20/20 [06:21<00:00, 19.07s/trial, best loss: -0.7770905250709086]
Наилучшие значения гиперпараметров {'C': 0.42219097013541806, 'penalty': 0, 'solver': 0}
F1 на тестовом наборе для логистической регресии (Hyperopt): 0.78


## Случайный лес

In [22]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 401, 1),
       'max_depth' : hp.quniform('max_depth', 5, 31, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=42)

    # обучаем модель
    # model.fit(X, y)
    # score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

trials_rf = Trials() # используется для логирования результатов

best = fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials_rf, # логирование результатов
          rstate=np.random.default_rng(42)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('F1 на тестовом наборе для случайного леса (Hyperopt): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

100%|██████████| 20/20 [01:20<00:00,  4.02s/trial, best loss: -0.8093103621614777]
Наилучшие значения гиперпараметров {'max_depth': 16.0, 'min_samples_leaf': 3.0, 'n_estimators': 327.0}
F1 на тестовом наборе для случайного леса (Hyperopt): 0.84


## Hyperopt — результаты

Удалось увеличить метрику F1 только для случайного леса.

# Optuna

## Логистическая регрессия

In [26]:
space ={
    'penalty': hp.choice('penalty', ['l2', 'none']) , # тип регуляризации
    'solver': hp.choice('solver', ['lbfgs', 'sag']), # алгоритм оптимизации
    'C': hp.uniform('C', 0.01, 10) # уровень силы регурялизации
} 

def optuna_rf(trial, cv=5):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l2', 'none'])
  solver = trial.suggest_categorical('solver', ['lbfgs', 'sag'])
  C = trial.suggest_float('C', 0.01, 10)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=42,
                                          max_iter=250)
  # обучаем модель
  # model.fit(X_train, y_train)
  # score = metrics.f1_score(y_train, model.predict(X_train))

  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("F1 на обучающем наборе для логистической регресии (Optuna): {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params, random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('F1 на тестовом наборе для логистической регресии (Optuna): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-04-15 17:23:21,882] A new study created in memory with name: LogisticRegression
[I 2023-04-15 17:23:30,359] Trial 0 finished with value: 0.7495325999076545 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 8.626094378336226}. Best is trial 0 with value: 0.7495325999076545.
[I 2023-04-15 17:23:43,812] Trial 1 finished with value: 0.7695615180137781 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 2.9385549469920997}. Best is trial 1 with value: 0.7695615180137781.
[I 2023-04-15 17:23:58,136] Trial 2 finished with value: 0.765192739560377 and parameters: {'penalty': 'none', 'solver': 'sag', 'C': 2.757930201780907}. Best is trial 1 with value: 0.7695615180137781.
[I 2023-04-15 17:24:06,436] Trial 3 finished with value: 0.7495325999076545 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 8.11554872079798}. Best is trial 1 with value: 0.7695615180137781.
[I 2023-04-15 17:24:21,501] Trial 4 finished with value: 0.765192739560377 and parameters: {'penalty': 

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'sag', 'C': 0.029763094055234793}
F1 на обучающем наборе для логистической регресии (Optuna): 0.79
F1 на тестовом наборе для логистической регресии (Optuna): 0.80


## Случайный лес

In [27]:
def optuna_rf(trial, cv=5):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 400, 10)
  max_depth = trial.suggest_int('max_depth', 5, 31, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)
  # обучаем модель
  # model.fit(X_train, y_train)
  # score = metrics.f1_score(y_train, model.predict(X_train))

  score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

  return score

# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("F1 на обучающем наборе для случайного леса (Optuna): {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('F1 на тестовом наборе для случайного леса (Optuna): {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

[I 2023-04-15 17:27:36,542] A new study created in memory with name: RandomForestClassifier
[I 2023-04-15 17:27:41,398] Trial 0 finished with value: 0.7758093634659164 and parameters: {'n_estimators': 360, 'max_depth': 7, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.7758093634659164.
[I 2023-04-15 17:27:52,282] Trial 1 finished with value: 0.8151297931724042 and parameters: {'n_estimators': 400, 'max_depth': 21, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.8151297931724042.
[I 2023-04-15 17:27:56,891] Trial 2 finished with value: 0.7758368668919158 and parameters: {'n_estimators': 350, 'max_depth': 7, 'min_samples_leaf': 10}. Best is trial 1 with value: 0.8151297931724042.
[I 2023-04-15 17:28:01,197] Trial 3 finished with value: 0.7895545749700418 and parameters: {'n_estimators': 280, 'max_depth': 8, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.8151297931724042.
[I 2023-04-15 17:28:05,703] Trial 4 finished with value: 0.8106817045906313 and parameters: {'n_es

Наилучшие значения гиперпараметров {'n_estimators': 220, 'max_depth': 28, 'min_samples_leaf': 2}
F1 на обучающем наборе для случайного леса (Optuna): 0.82
F1 на тестовом наборе для случайного леса (Optuna): 0.83


## Optuna — выводы

Удалось увеличить метрику F1 только для логистической регресии.

# Общие результаты

С помощью методов RandomizedSearchCV и Optuna нам удалось увеличить метрику F1 для логистической регресии по сравнению с базовой моделью. С помощью метода Hyperopt нам удалось увеличить метрику F1 для случайного леса по сравнению с базовой моделью.